# Classic Tic Tac Toe using Reinforcement Learning

### Importing modules
1. `random`: To generate epsilon (exploration vs exploitation)
2. `json`: To store the state space in a json file

In [14]:
import random
import json

### Defining environment

In [15]:
class environment:
    
    def __init__(self):
        self.state = ['-' for i in range(9)]
        self.symbol_mapping = {'-': 0,'X': 1,'O': 2}
        self.state_value = {}
    
    def display(self):
        '''
        Displaying the 3x3 grid
        '''
        for row in range(3):
            for col in range(3):
                print("{}".format(self.state[3*row+col]), end=' ')
            print()
            
    def state_hash(self, state_matrix):
        '''
        Calculating the hash value of a state
        '''
        hash_val = 0
        for val in state_matrix:
            hash_val = 3*hash_val + self.symbol_mapping[val]
        return hash_val
    
    def list_empty(self):
        '''
        Finding all the empty spaces in game
        '''
        empty_index = []
        for ind,val in enumerate(self.state):
            if val == '-':
                empty_index.append(ind)
        return empty_index
    
    def end_game(self):
        '''
        Returns True/False depending on if there is empty spaces
        '''
        return len(self.list_empty())==0
    
    def isWon(self, symbol):
        '''
        Checks the winning condition in game
        '''
        for (ind1, ind2, ind3) in [(0,1,2), (3,4,5), (6,7,8), (0,3,6), (1,4,7), (2,5,8), (0,4,8), (2,4,6)]:
            if(self.state[ind1]==symbol and self.state[ind2]==symbol and self.state[ind3]==symbol):
                return True
        return False
    
    def update_state_value(self, agent_hash, reward):
        '''
        Updates the state value according to Reinforcement Learning
        '''
        gamma = 0.8
        for ind,val in enumerate(agent_hash[::-1]):
            state_reward,n = self.state_value.setdefault(val, (1,0))
            self.state_value[val] = ((state_reward*n + reward)/(n+1), n+1)
            reward *= gamma

In [16]:
e = environment()
e.display()
print(e.state_hash(e.state))
print(e.list_empty())
print(e.end_game())
print(e.isWon('O'))
e.update_state_value([6,5,4],1)
print(e.state_value)

- - - 
- - - 
- - - 
0
[0, 1, 2, 3, 4, 5, 6, 7, 8]
False
False
{4: (1.0, 1), 5: (0.8, 1), 6: (0.6400000000000001, 1)}


### Defining agent

In [17]:
class agent:
    
    def __init__(self, symbol):
        self.symbol = symbol
        self.state_hash_stack = []
#         self.mapping = {'-': 0, 'X': 1, 'O': 2}
    
    def action(self, environment):
        '''
        Determines the action to be taken by the agent
        '''
        temp_possible_state = []
        
        # Agent checks for all empty space and gets indexs
        for ind in environment.list_empty():
            temp_state = environment.state.copy()
            temp_state[ind] = self.symbol
            
            # Find the corresponding hash values
            temp_hash_state_value = environment.state_hash(temp_state)
            temp_possible_state.append((ind, temp_hash_state_value, environment.state_value.setdefault(temp_hash_state_value, (1,0))))
            
        # Finds argmax of possiblilty
        if random.random() < 0.9:
            optimal_action = max(temp_possible_state, key=lambda x: x[2][0])
        else:
            optimal_action = random.sample(temp_possible_state, 1)[0]
        
        # Puts in stack
        self.state_hash_stack.append(optimal_action[1])
        return optimal_action[0]

In [18]:
a = agent('X')
print(a.action(e))
e.state_value
print(a.state_hash_stack)

0
[6561]


### Training for game

In [19]:
def main():
    # Initialising the agent and environment
    env = environment()
    agentO = agent('O')
    agentX = agent('X')
    
    # 'X' always plays first and t_game is number of games to be played
    agent_mapping = {0: agentX, 1: agentO}
    chance = 0
    t_game = 100_000
    
    # Iterating for each game
    for n_games in range(t_game):
        
        # Initialising for each game
        env.state = ['-' for i in range(9)]
        agent_mapping[chance].state_hash_stack = []
        agent_mapping[chance^1].state_hash_stack = []
        chance = 0
                                      
        while not env.end_game():
            # Calculating and updating the move by agent
            move = agent_mapping[chance].action(env)
            env.state[move] = agent_mapping[chance].symbol
            
            # Displaying the last sample game
            if n_games == t_game -1:
                print(env.state)
                env.display()
                
            # Winning condition, RL updates the state value of environment
            if env.isWon(agent_mapping[chance].symbol):
                env.update_state_value(agent_mapping[chance].state_hash_stack,1)
                env.update_state_value(agent_mapping[chance^1].state_hash_stack,-1)
                break
                
            # Opponent's turn
            chance ^= 1
            
        # If the game is draw, RL updates the state value of environment
        if not (env.isWon(agent_mapping[chance].symbol) and env.isWon(agent_mapping[chance^1].symbol)):
            env.update_state_value(agent_mapping[chance].state_hash_stack,0)
            env.update_state_value(agent_mapping[chance^1].state_hash_stack,0)
        
        # Displaying the status after every 500 games played
        if n_games%500 == 0:
            print("STATUS: Total games played:{} and length of state space: {}".format(n_games+500,len(env.state_value)))
    
    # Storing the result state space in JSON
    with open("State_space.json", "w") as file:
        json.dump(env.state_value, file)
        
main()

STATUS: Total games played:500 and length of state space: 42
STATUS: Total games played:1000 and length of state space: 2692
STATUS: Total games played:1500 and length of state space: 3203
STATUS: Total games played:2000 and length of state space: 3711
STATUS: Total games played:2500 and length of state space: 3966
STATUS: Total games played:3000 and length of state space: 4162
STATUS: Total games played:3500 and length of state space: 4550
STATUS: Total games played:4000 and length of state space: 4703
STATUS: Total games played:4500 and length of state space: 4783
STATUS: Total games played:5000 and length of state space: 4833
STATUS: Total games played:5500 and length of state space: 4870
STATUS: Total games played:6000 and length of state space: 4924
STATUS: Total games played:6500 and length of state space: 4944
STATUS: Total games played:7000 and length of state space: 4957
STATUS: Total games played:7500 and length of state space: 4993
STATUS: Total games played:8000 and length 

STATUS: Total games played:64000 and length of state space: 5470
STATUS: Total games played:64500 and length of state space: 5470
STATUS: Total games played:65000 and length of state space: 5470
STATUS: Total games played:65500 and length of state space: 5470
STATUS: Total games played:66000 and length of state space: 5470
STATUS: Total games played:66500 and length of state space: 5470
STATUS: Total games played:67000 and length of state space: 5472
STATUS: Total games played:67500 and length of state space: 5472
STATUS: Total games played:68000 and length of state space: 5472
STATUS: Total games played:68500 and length of state space: 5472
STATUS: Total games played:69000 and length of state space: 5473
STATUS: Total games played:69500 and length of state space: 5473
STATUS: Total games played:70000 and length of state space: 5473
STATUS: Total games played:70500 and length of state space: 5473
STATUS: Total games played:71000 and length of state space: 5473
STATUS: Total games playe

### Defining User

In [20]:
class User:

    def __init__(self, symbol):
        self.symbol = symbol
        
    def action(self, env):
        '''
        Determines the action to be taken by the user.
        Valid input 1-9
        '''
        while not env.end_game():
            try:
                data = input("Enter position\n")
                if data == 'q' or data == 'Q':
                    return -1
                else:
                    data = int(data)
                if data in range(1,10):
                    index = 3*(data//3) + (data%3) -1
                    if index in env.list_empty():
                        return index
                    else:
                        print("Index is taken")
                else:
                    print("Index out of bound")
                
            except:
                print("Enter 2-digit number [0-2][0-2]")

In [21]:
user = User('O')
user.action(e)

Enter position
q


-1

### Defining trained RL model

In [22]:
class trained_agent:
    
    def __init__(self, symbol):
        self.symbol = symbol
        with open("State_space.json", "r") as file:
            self.state_value = json.load(file)
        print(len(self.state_value))
    
    def action(self, environment):
        '''
        Determines the action to be taken by RL trained model
        '''
        temp_possible_state = []
        
        # Agent checks for all empty space and gets indexs
        for ind in environment.list_empty():
            temp_state = environment.state.copy()
            temp_state[ind] = self.symbol
            
            # Find the corresponding hash values
            temp_hash_state_value = environment.state_hash(temp_state)
            temp_possible_state.append((ind, self.state_value.setdefault(str(temp_hash_state_value), (1,0))))
            
        # Finds argmax of possiblilty
        optimal_action = max(temp_possible_state, key=lambda x: x[1][0])
        
        return optimal_action[0]

In [23]:
t = trained_agent('X')
t.action(e)

5474


4

### User VS RL Trained model

In [28]:
def user_game():
    # Defining the environment
    env = environment()
    
    # Giving the user option to choose 'X' or 'O'
    while True:
        try:
            XO = int(input("Enter 0 for O and 1 for X"))
            print(XO, type(XO))
            if XO == 0:
                user = User('O')
                agentX = trained_agent('X')
                agent_mapping = {0: agentX, 1: user}
                break
            elif XO == 1:
                user = User('X')
                agentO = trained_agent('O')
                agent_mapping = {0: user, 1: agentO}
                break
            else:
                print("Try again")
        except:
            print("Wrong input")
    
    # 'X' always plays first
    chance = 0
    
    while not env.end_game():
        # Finding the action to be played
        move = agent_mapping[chance].action(env)
        if move == -1:
            print("User loses, computer wins")
            return
        env.state[move] = agent_mapping[chance].symbol
        
        # Displaying the board state
        print("{} move".format(agent_mapping[chance].symbol))
        env.display()
        
        # Winning condition
        if env.isWon(agent_mapping[chance].symbol):
            print("{} wins".format(agent_mapping[chance].symbol))
            break
            
        # Opponent's turn
        chance ^= 1
    
user_game()

Enter 0 for O and 1 for X0
0 <class 'int'>
5474
X move
- - - 
- X - 
- - - 
Enter position
2
O move
- O - 
- X - 
- - - 
X move
- O X 
- X - 
- - - 
Enter position
7
O move
- O X 
- X - 
O - - 
X move
- O X 
- X X 
O - - 
Enter position
4
O move
- O X 
O X X 
O - - 
X move
- O X 
O X X 
O - X 
X wins


# Status
User as 'x' plays first:  
at corner: draw  
at center: draw  
at edge: draw  

User as 'o' plays second:  
at corner: draw  
at center: nil  
at edge: loses  